In [1]:
import pandas as pd
import numpy  as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import spacy

In [2]:
data_origin = pd.read_csv("train.csv")
data_test = pd.read_csv("test.csv")

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline

In [4]:
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Load English tokenizer, tagger, parser, NER and word vectors
parser = English()

# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(sentence)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [5]:
# Custom transformer using spaCy
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        # Cleaning Text
        return [clean_text(text) for text in X]

    def fit(self, X, y=None, **fit_params):
        return self

    def get_params(self, deep=True):
        return {}

# Basic function to clean the text
def clean_text(text):
    # Removing spaces and converting text into lowercase
    return text.strip().lower()

In [6]:
bow_vector = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1))

In [7]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [8]:
from sklearn.model_selection import train_test_split

X = data_origin["description"]# the features we want to analyze
ylabels = data_origin["jobflag"] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.25,stratify=data_origin.jobflag)

In [19]:
train_count = bow_vector.fit_transform(X_train)

In [20]:
train_tfidf = tfidf_vector.fit_transform(X_train)

In [21]:
train_tfidf

<2198x3651 sparse matrix of type '<class 'numpy.float64'>'
	with 21899 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn.feature_selection import SelectKBest
hoge = SelectKBest(k=2000).fit_transform(train_tfidf,y_train)

In [24]:
hoge.shape

(2198, 2000)

In [25]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import time

In [26]:
params ={
    "C":np.arange(0.1,2,0.05),
    "kernel":["linear"]
}
grid = GridSearchCV(SVC(),params,cv=6,n_jobs=6)

In [27]:
start = time.time()
grid.fit(hoge,y_train)
print("time",time.time()-start)

time 26.714841604232788


In [28]:
grid.best_params_

{'C': 1.4500000000000006, 'kernel': 'linear'}

In [31]:
test_count = count_vect.transform(X_test)
test_tfidf = tfidf_trans.transform(test_count)

In [34]:
test_tfidf = SelectKBest(k=2000).fit(test_tfidf)

TypeError: fit() missing 1 required positional argument: 'y'

In [33]:
pred = grid.predict(hoge)
print(classification_report(y_train,pred))
pred = grid.predict(test_tfidf)
print(classification_report(y_train,pred))

             precision    recall  f1-score   support

          1       0.85      0.85      0.85       468
          2       0.93      0.60      0.73       261
          3       0.83      0.96      0.89      1032
          4       0.94      0.78      0.85       437

avg / total       0.87      0.86      0.86      2198



ValueError: X.shape[1] = 4154 should be equal to 2000, the number of features at training time

In [23]:
print("default")
pred = grid.predict(input_train)
print(classification_report(output_train,pred))
pred = grid.predict(input_test)
print(classification_report(output_test,pred))

default
             precision    recall  f1-score   support

          1       0.88      0.89      0.89       499
          2       0.95      0.69      0.80       278
          3       0.87      0.98      0.92      1101
          4       0.95      0.82      0.88       466

avg / total       0.90      0.89      0.89      2344

             precision    recall  f1-score   support

          1       0.67      0.59      0.63       125
          2       0.62      0.34      0.44        70
          3       0.68      0.88      0.77       275
          4       0.70      0.49      0.58       117

avg / total       0.68      0.68      0.66       587



In [36]:
test_count = count_vect.transform(data_test.description)
test_tfidf = tfidf_trans.transform(test_count)

In [37]:
pred_sub = grid.predict(test_tfidf)
data_test['pre'] = pred_sub
data_test.to_csv('submit_svm_wordvect1.csv',columns=['id','pre'],header=False,index=False)

In [99]:
print(data_origin.description[data_origin["jobflag"] == 2])

0       Executes and writes portions of testing plans,...
10      Build scalable machine learning services and d...
13      Work with subject matter experts to gather req...
23      Recognized as a technical expert within the On...
39      Inform bidding strategy and data engineering a...
42      Demonstrate the potential to initiate, lead, a...
59      Derivation of optimal software architecture fo...
60      Work closely with Research Scientist team memb...
74      Works with other machine engineering leads to ...
75                   Defining data augmentation pipelines
83      Flexible to travel to other SEI offices in Pit...
87      Work with cloud and data architects to define ...
106     Learn about machine learning, data science, co...
116     Plan and build data-driven banking ecosystem, ...
128     Infrastructure provisioning and integration wi...
130              Build API interface for AI/Client models
133     Training high quality domain-specific models o...
137     Comple